In [1]:
import pandas as pd
import numpy as np

In [2]:
df_final = pd.read_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2_no_stock_no_stat_filter_20210226.csv')

In [3]:
print(df_final.head())

   Unnamed: 0   item_code  avg_conv_rate  last_price   last_date  \
0           0  1000012530       0.016220       99.99  2021-02-13   
1           1  1000012794       0.014916      259.99  2021-02-25   
2           3  1000022179       0.003121      479.99  2021-02-06   
3           4  1000024320       0.024532      119.99  2020-12-01   
4           5  1000034743       0.005481      279.99  2021-02-04   

   avg_unique_views      PE  PE_err  has_enough_stat      pc2  ...  \
0         66.081387 -0.0187  0.0059             True   44.680  ...   
1         41.525974 -0.0120  0.0052            False  141.050  ...   
2         93.258739 -0.0136  0.0033             True  276.870  ...   
3         22.354839 -0.0220  0.0097            False   65.109  ...   
4        110.202494 -0.0186  0.0048             True  171.030  ...   

   price_down  conv_rate_down  conv_rate_down_err  Delta_price  Margin_down  \
0   98.010065        0.016226        1.914621e-06    39.345210    42.186330   
1  257.40305

In [4]:
df_final = df_final[['item_code','last_price','price_down','Margin_rate_to_base','PE','PE_err','has_enough_stat','pc2']]

In [5]:
#df_final['Margin_rate_to_base']=df_final['Margin_rate_to_base']*100

In [6]:

df_final.head()

,item_code,last_price,price_down,Margin_rate_to_base,PE,PE_err,has_enough_stat,pc2
0,1000012530,99.99,98.010065,0.000374,-0.0187,0.0059,True,44.680
1,1000012794,259.99,257.403056,0.000120,-0.0120,0.0052,False,141.050
2,1000022179,479.99,470.485561,0.000272,-0.0136,0.0033,True,276.870
3,1000024320,119.99,118.796080,0.000220,-0.0220,0.0097,False,65.109
4,1000034743,279.99,274.445827,0.000372,-0.0186,0.0048,True,171.030


In [7]:
df_final=df_final.rename(columns={"last_price": "actual_price", "price_down": "optimal_price"})

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('read_tables').getOrCreate()


In [9]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [10]:
df_d_item.createOrReplaceTempView('d_item')

In [11]:
df_d_item.show()

+---------------+------------------+---------------------+-------------+----------------+----------+-------------+-----------+----------+-----------+--------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+------------------+-------------------+-------------------+---------+--------------------------------+---------------------------------------------+-----------------------+---------------------+------------------------+--------------------------+------------------------+--------------------+-----------+-------------------+------------------+-------------------+----------------------------+---------------+-----------------------+-----------------------+----------------------+------------------------------------+-------------------------------------------+-------------------------------------------------+--------------------------------------------------------+------------------------+---------------------+---

In [12]:
df_spark_final = spark.createDataFrame(df_final)


In [13]:
df_spark_final.createOrReplaceTempView('margin')

In [14]:
sp_df_pc2 = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_item_status_last/*.parquet',
    header=True)
sp_df_pc2.show()

+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
|  item_skey|shop_skey|item_status_skey|item_status_weekly_skey|item_pc1_costs_before_cogs|item_pc2_costs_before_cogs|group_pc2_margin_simulated|group_pc2_margin_ratio_simulated|group_pc2_margin_simulated_pocket|group_pc2_margin_ratio_simulated_pocket|
+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
| 4184608796|        1|               4|                      4|                   674.169|                   800.340|                      null|                            null|                             null|                             

In [15]:
sp_df_pc2.createOrReplaceTempView('f_item_last_status')

In [16]:
df_d_item_table_margin = spark.sql("""SELECT    margin.*,
                                                di.item_parent_item_code, di.item_main_category, di.item_inventory_flag, di.item_gtin_comparable,di.item_exclusivity,di.item_skey,
                                                fils.item_pc1_costs_before_cogs as pc1_costs
                                                from margin
                                                INNER JOIN d_item as di on di.item_code = margin.item_code
                                                INNER JOIN f_item_last_status as fils on fils.item_skey=di.item_skey and fils.shop_skey=1
                                                """)

df_d_item_table_margin.show()

+----------+------------+------------------+--------------------+--------------------+------+---------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|                  PE|PE_err|has_enough_stat|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|
+----------+------------+------------------+--------------------+--------------------+------+---------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
|1000012530|       99.99| 98.01006534394246|3.740000000003154E-4|             -0.0187|0.0059|           true|             44.68|   000000008000000774|ESSZIMMERSTÜHLE&B...|               laso|      non-comparable|       exclusive|247

In [17]:
df_d_item_table_margin.createOrReplaceTempView('item_margin')

In [18]:
df_standalone_item = spark.sql("""SELECT *
                from item_margin
                where item_parent_item_code like '%-P' """)

#127 items

In [19]:
df_pd_sa_item=df_standalone_item.toPandas()



In [20]:
df_pd_sa_item['item_code'].head()


0    1000125896
1    1000007327
2    1000007822
3    1000008870
4    1000117179
Name: item_code, dtype: int64

In [21]:
item_sa_list = list(df_pd_sa_item['item_code'])

In [22]:
print(item_sa_list)

[1000125896, 1000007327, 1000007822, 1000008870, 1000117179, 1000123635, 1000151797, 1000166801, 1000168702, 1000178079, 1000183575, 1000183576, 1000194074, 1000216767, 1000218264, 1000050845, 1000052582, 1000064521, 1000129145, 1000166814, 1000174312, 1000174905, 1000180146, 1000008827, 1000027423, 1000032520, 1000053564, 1000092132, 1000104023, 1000149139, 1000156992, 1000182807, 1000199454, 1000209294, 1000215684, 1000007797, 1000012742, 1000119830, 1000123493, 1000137900, 1000166806, 1000167289, 1000007325, 1000065698, 1000145838, 1000153506, 1000189194, 1000207475, 1000211591, 1000053245, 1000054183, 1000087134, 1000149709, 1000007079, 1000052646, 1000063634, 1000064530, 1000085424, 1000092325, 1000104096, 1000111508, 1000117706, 1000118225, 1000225830, 1000008834, 1000052641, 1000053479, 1000117722, 1000123698, 1000125121, 1000126096, 1000126858, 1000008124, 1000008790, 1000015709, 1000051350, 1000118214, 1000176576, 1000182985, 1000194075, 1000008048, 1000031031, 1000051675, 100

In [23]:
pd_d_item_margin = df_d_item_table_margin.toPandas()




In [24]:
import psycopg2

def connect_redshift_db():
    cnx = psycopg2.connect(dbname= 'dwh',
                     host='redshift-prod.data.service.home24.net',
                     port= '5439',
                     user= 'gabriele_sabato',
                     password= 'zQZ8x6ZhRoTf7GS')
    return cnx
cnx_redshift = connect_redshift_db()
cursor_redshift = cnx_redshift.cursor()

In [25]:
def query_to_dataframe(_cursor, _query):
    _cursor.execute(_query)
    colnames = [desc[0] for desc in _cursor.description]
    print(colnames)
    return pd.DataFrame(_cursor.fetchall(), columns=colnames)

In [26]:
item_list=list(pd_d_item_margin['item_code'])
new_list = []
final_list = ""

for i in item_list:
    new_list.append(" '00000000" + str(i)+"' , " )

print(new_list)
final_list = final_list.join(new_list)
final_list=final_list[0:-3]



[" '000000001000012530' , ", " '000000001000012794' , ", " '000000001000022179' , ", " '000000001000024320' , ", " '000000001000034743' , ", " '000000001000034762' , ", " '000000001000119742' , ", " '000000001000125896' , ", " '000000001000127678' , ", " '000000001000145006' , ", " '000000001000161399' , ", " '000000001000199717' , ", " '000000001000200909' , ", " '000000001000005727' , ", " '000000001000007327' , ", " '000000001000007822' , ", " '000000001000008870' , ", " '000000001000016028' , ", " '000000001000016120' , ", " '000000001000016194' , ", " '000000001000022565' , ", " '000000001000050787' , ", " '000000001000104353' , ", " '000000001000110756' , ", " '000000001000114877' , ", " '000000001000116614' , ", " '000000001000117179' , ", " '000000001000123635' , ", " '000000001000128764' , ", " '000000001000151797' , ", " '000000001000159193' , ", " '000000001000166801' , ", " '000000001000168702' , ", " '000000001000175476' , ", " '000000001000176734' , ", " '0000000010001780

In [27]:
print(final_list)

 '000000001000012530' ,  '000000001000012794' ,  '000000001000022179' ,  '000000001000024320' ,  '000000001000034743' ,  '000000001000034762' ,  '000000001000119742' ,  '000000001000125896' ,  '000000001000127678' ,  '000000001000145006' ,  '000000001000161399' ,  '000000001000199717' ,  '000000001000200909' ,  '000000001000005727' ,  '000000001000007327' ,  '000000001000007822' ,  '000000001000008870' ,  '000000001000016028' ,  '000000001000016120' ,  '000000001000016194' ,  '000000001000022565' ,  '000000001000050787' ,  '000000001000104353' ,  '000000001000110756' ,  '000000001000114877' ,  '000000001000116614' ,  '000000001000117179' ,  '000000001000123635' ,  '000000001000128764' ,  '000000001000151797' ,  '000000001000159193' ,  '000000001000166801' ,  '000000001000168702' ,  '000000001000175476' ,  '000000001000176734' ,  '000000001000178079' ,  '000000001000183575' ,  '000000001000183576' ,  '000000001000185455' ,  '000000001000194074' ,  '000000001000199674' ,  '00000000100021

In [28]:
campaign_flag = """SELECT item_code,item_campaign_category, item_campaign_key, item_campaign_label
                    FROM dwh.public.item_catalog
                    WHERE item_code in ({FINAL_LIST})
                        and app_domain_id=1 """.format(FINAL_LIST=final_list)



In [29]:
df_campaign = query_to_dataframe(cursor_redshift, campaign_flag)



['item_code', 'item_campaign_category', 'item_campaign_key', 'item_campaign_label']


In [30]:
df_campaign.head()

,item_code,item_campaign_category,item_campaign_key,item_campaign_label
0,000000001000005134,,,
1,000000001000005777,,,
2,000000001000005392,,,
3,000000001000005839,,,
4,000000001000005844,,,


In [31]:
item_skey_list = list(pd_d_item_margin['item_skey'])
string_ints = [str(int) for int in item_skey_list]
str_of_ints = ",".join(string_ints)
print(str_of_ints)



247523697279378,151936474932722,11954970507506,31827848594342,206871238538442,192085517288465,118864457774015,168012620788218,10396424081065,280637889289282,146164006422056,126915281894573,185007068719582,71488185050378,9591881119337,224785681040185,242371880591990,48800168481821,55952455697067,58103166898131,24924066511817,50327313240902,207313610411423,200487217084832,25038272605281,263318503441531,34780482897946,120701856307535,231881185838527,272193278534478,243018050005606,167211492608660,7452747415115,115260611693891,38255991794505,275546052781316,65906826950238,21687571143275,143465991346429,242522860992899,199375335714241,148474842384330,160187312950916,215803683932252,84087042756332,66003517992621,55118908543253,205605370420399,229270075861196,71232050084551,69849851848335,72171059527105,55393597474959,92807562060189,14345875489892,168504712280492,249510402315032,145551551942177,157052607139695,262990028891641,181192477412169,8949408463673,76926321591939,129711289847006,244365

In [32]:
last_price_rule = """
SELECT item_skey, item_final_price_rule, period_date_skey--item_final_price_rule
  from dwh.star_analytical.f_item_attribute_history
 where shop_skey = 1
   and period_date_skey = (SELECT MAX(period_date_skey)
                             FROM dwh.star_analytical.f_item_attribute_history)
   and item_skey in ({ITEM_SKEY})
""".format(ITEM_SKEY=str_of_ints)

In [33]:
df_last_price_rule = query_to_dataframe(cursor_redshift, last_price_rule)

['item_skey', 'item_final_price_rule', 'period_date_skey']


In [34]:
sp_last_price_rule = spark.createDataFrame(df_last_price_rule)
sp_last_price_rule.createOrReplaceTempView('last_price_rule')
sp_campaign = spark.createDataFrame(df_campaign)
sp_campaign.createOrReplaceTempView('campaign')

In [35]:
final_df_margin = spark.sql("""SELECT im.*,
                    lpr.item_final_price_rule,
                    camp.item_campaign_key,
                    camp.item_campaign_label
                    from item_margin as im
                    INNER JOIN last_price_rule as lpr on lpr.item_skey = im.item_skey
                    INNER JOIN campaign as camp on CAST(camp.item_code as int) = im.item_code
                    """)

In [36]:
#final_df_margin.show()
final_df_margin.createOrReplaceTempView("margin_campaign")

In [37]:
OOS_query = """
  WITH fc_dates as (
      SELECT *,
             RIGHT(week_iso, 2)::int                                                                    as calendar_week,
             SUBSTRING(week_iso, LENGTH(week_iso) - 7, 4)::int                                          as calendar_year,
             cast(SUBSTRING(week_iso, LENGTH(week_iso) - 7, 4)::text + RIGHT(week_iso, 2)::text as int) as year_week
        FROM forecasting_ext.supply_chain_simulation_latest
  ),
       next_incoming_date as (
           SELECT item_code,
                  MIN(year_week) as next_incoming_week
             FROM fc_dates
            WHERE incoming_total > 0
            GROUP BY 1
       ),
       next_incoming as (
           SELECT fc.item_code,
                  incoming_total as next_incoming_qty
             FROM fc_dates fc
                  LEFT JOIN next_incoming_date
                  on fc.item_code = next_incoming_date.item_code
            WHERE next_incoming_week = year_week
       ),
       tmp_variables AS (
           SELECT week_iso
             FROM star_analytical.d_calendar
            WHERE date = CAST(GETDATE() AS DATE)
       )
SELECT fc.forecast_creation_date,
       fc.item_code,
       di.item_skey,
       fc.inventory_flag,
       fc.item_supplier_location,
       case
           when fc.inco_terms = 'FOB'
               then 'Import'
           when fc.item_supplier_location like '%Import%'
               then 'Import'
           else fc.item_supplier_location
           end                                          as item_sourcing_location,
       fc.item_supplier_regular_id,
       di.item_supplier_regular,
       CASE
           WHEN di.item_supplier_lead_time <= 0
               THEN NULL
           ELSE di.item_supplier_lead_time END          AS item_supplier_lead_time,
       fc.inco_terms,
       di.item_cross_distribution_chain_material_status as distribution_status,
       di.item_cross_plant_material_status              as material_status,
       fc.item_main_category,
       next_incoming_week,
       next_incoming_qty,
       sum(incoming_total)                              as total_incoming,
       MAX(CASE
               WHEN week_iso = (SELECT week_iso
                                  FROM tmp_variables) AND dispo_stock_quantity <= 0
                   THEN 1
               ELSE 0 END)                              AS currently_out_of_stock,
       MAX(CASE
               WHEN week_iso = (SELECT week_iso
                                  FROM tmp_variables)
                   THEN floor(dispo_stock_quantity)
               ELSE NULL END)                           AS current_stock_level,
       sum(CASE
               WHEN fc.dispo_stock_quantity <= 0
                   THEN 1
               ELSE 0 END)                              AS number_of_weeks_out_of_stock_until_leadtime,
       sum(CASE
               WHEN sales_quantity_forecasted = 0
                   THEN 1
               ELSE 0 END)                              AS number_of_weeks_not_sellable_until_leadtime
  FROM fc_dates fc
       LEFT JOIN next_incoming_date
       on fc.item_code = next_incoming_date.item_code
       LEFT JOIN next_incoming
       on fc.item_code = next_incoming.item_code
       LEFT JOIN star_analytical.d_item di
       ON fc.item_code = di.item_code
  WHERE item_skey in ({ITEM_SKEY})
 GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,15
""".format(ITEM_SKEY=str_of_ints)

df_OOS = query_to_dataframe(cursor_redshift, OOS_query)

sp_OOS = spark.createDataFrame(df_OOS)
sp_OOS.createOrReplaceTempView('oos_table')
sp_OOS.show()

['forecast_creation_date', 'item_code', 'item_skey', 'inventory_flag', 'item_supplier_location', 'item_sourcing_location', 'item_supplier_regular_id', 'item_supplier_regular', 'item_supplier_lead_time', 'inco_terms', 'distribution_status', 'material_status', 'item_main_category', 'next_incoming_week', 'next_incoming_qty', 'total_incoming', 'currently_out_of_stock', 'current_stock_level', 'number_of_weeks_out_of_stock_until_leadtime', 'number_of_weeks_not_sellable_until_leadtime']
+----------------------+------------------+---------------+--------------+----------------------+----------------------+------------------------+---------------------+-----------------------+----------+-------------------+---------------+------------------+------------------+-----------------+----------------+----------------------+-------------------+-------------------------------------------+-------------------------------------------+
|forecast_creation_date|         item_code|      item_skey|inventory_fla

In [38]:
margin_oos = spark.sql("""
            SELECT mc.*,
            oos.currently_out_of_stock as currently_oos,
            oos.number_of_weeks_out_of_stock_until_leadtime as weeks_oos_leadtime,
            oos.current_stock_level as available_stock_qty
            from margin_campaign as mc
            INNER JOIN oos_table as oos on oos.item_skey=mc.item_skey """)

margin_oos.show()

+----------+------------+------------------+--------------------+--------------------+--------------------+---------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+---------------------+-----------------+--------------------+-------------+------------------+-------------------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|                  PE|              PE_err|has_enough_stat|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|item_final_price_rule|item_campaign_key| item_campaign_label|currently_oos|weeks_oos_leadtime|available_stock_qty|
+----------+------------+------------------+--------------------+--------------------+--------------------+---------------+------------------+---------------------+--------------------+-------------------+--------------------+------

In [39]:
margin_oos.createOrReplaceTempView('margin_oos_table')

In [40]:
#Calculate price median and quantity sold
df_median_and_sales = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top5k_seller_YTD_median_price_tot_sales_20210226.pickle')

In [41]:
sp_median_sales = spark.createDataFrame(df_median_and_sales)

In [42]:
sp_median_sales.createOrReplaceTempView('median_price_qty_sales')

In [43]:
sp_median_sales.show()

+---------------+-----------------+-------------+
|      item_skey|median_price_LYTD|qty_sold_LYTD|
+---------------+-----------------+-------------+
| 71940907335782|      74.98999999|          345|
|177867612436569|           134.99|          239|
|130092928900034|           649.99|           52|
|266312525233491|            49.99|          404|
|173156703372281|           399.99|          521|
|101962757279480|           249.99|          140|
|231742782865803|      99.98999999|         1054|
|122535632769235|           349.99|          136|
|172969960437177|           249.99|          172|
|106266744563234|      89.98999999|          500|
| 32288152001464|           299.99|          203|
|140539805545547|           199.99|          106|
|109999215338205|           249.99|          347|
|142441836666160|           104.99|          267|
| 74462021394914|            34.99|         1393|
|245742233824797|           329.99|           59|
|174223993968068|           179.99|          188|


In [44]:
final_df = spark.sql("""SELECT moos.*,
                        ABS((moos.PE_err)/(moos.PE)) as rel_error,
                        mpqs.median_price_LYTD,
                        mpqs.qty_sold_LYTD
                        FROM margin_oos_table as moos
                        INNER JOIN median_price_qty_sales as mpqs on moos.item_skey = mpqs.item_skey""")


In [45]:
final_df.show()

+----------+------------+------------------+--------------------+--------------------+--------------------+---------------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+---------------------+-----------------+--------------------+-------------+------------------+-------------------+-------------------+-----------------+-------------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|                  PE|              PE_err|has_enough_stat|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|item_final_price_rule|item_campaign_key| item_campaign_label|currently_oos|weeks_oos_leadtime|available_stock_qty|          rel_error|median_price_LYTD|qty_sold_LYTD|
+----------+------------+------------------+--------------------+--------------------+--------------------+---------------+-----

In [46]:
df_margin_panda= final_df.toPandas()


In [47]:

df_margin_panda.to_csv('../raw_data/Excel_files/Margin_result_no_filter_all_stat_all_in_complete_20210226.csv')